In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [3]:
df=pd.read_csv(r"C:\Users\Admin\Downloads\uber_rides_data.xlsx - sample_train.csv")

In [54]:
df

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...,...
199995,42598914,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
199996,16382965,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
199997,27804658,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
199998,20259894,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695415,1


In [55]:
df.shape

(200000, 8)

In [56]:
df.dtypes

ride_id                int64
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [57]:
df["dropoff_longitude"].isna().sum()

1

In [58]:
df.isna().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64

In [59]:
df.dropna(inplace=True)

In [60]:
df.isna().sum()

ride_id              0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

In [61]:
average_fare = df['fare_amount'].mean()
print(average_fare)

11.359891549457748


In [62]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two sets of latitude and longitude coordinates.
    """
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c

    return distance

In [63]:
df.columns

Index(['ride_id', 'fare_amount', 'pickup_datetime', 'pickup_longitude',
       'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
       'passenger_count'],
      dtype='object')

In [64]:
df['haversine_distance'] = df.apply(lambda row: haversine_distance(row['pickup_latitude'], row['pickup_longitude'],row['dropoff_latitude'], row['dropoff_longitude']), axis=1)
median_distance = df['haversine_distance'].median()
print("Median Haversine Distance:", median_distance)

Median Haversine Distance: 2.120992396182902


In [65]:
df['haversine_distance'].max()

16409.23913531317

In [66]:
count_zero_distance_rides = (df['haversine_distance'] == 0).sum()
print("Number of rides with 0.0 Haversine Distance:", count_zero_distance_rides)


Number of rides with 0.0 Haversine Distance: 5632


In [67]:
zero_distance_rides = df[df['haversine_distance'] == 0.0]
mean_fare_for_zero_distance = zero_distance_rides['fare_amount'].mean()
print("Mean 'fare_amount' for rides with 0.0 Haversine Distance:", mean_fare_for_zero_distance)


Mean 'fare_amount' for rides with 0.0 Haversine Distance: 11.585317826704546


In [68]:
df["fare_amount"].max()

499.0

In [69]:
df[["pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude"]].isna().sum()

pickup_latitude      0
pickup_longitude     0
dropoff_latitude     0
dropoff_longitude    0
dtype: int64

In [70]:
extreme_values = df[(df['pickup_longitude'] < -180) | (df['pickup_longitude'] > 180) |
                    (df['pickup_latitude'] < -90) | (df['pickup_latitude'] > 90) |
                    (df['dropoff_longitude'] < -180) | (df['dropoff_longitude'] > 180) |
                    (df['dropoff_latitude'] < -90) | (df['dropoff_latitude'] > 90)]
extreme_values.reset_index()

,index,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_distance
0,4949,42931506,4.9,2012-04-28 00:58:00 UTC,-748.016667,40.739957,-74.003570,40.734192,1,3828.760217
1,32549,5634081,15.7,2012-06-16 10:04:00 UTC,-74.016055,40.715155,-737.916665,40.697862,2,4644.267681
2,48506,1055960,33.7,2011-11-05 23:26:00 UTC,-735.200000,40.770092,-73.980187,40.765530,1,4852.352520
3,56617,14257861,8.1,2012-03-11 07:24:00 UTC,-73.960828,404.433332,-73.988357,40.769037,1,407.457236
4,61793,2849369,8.5,2012-06-13 05:45:00 UTC,-73.951385,401.066667,-73.982110,40.754117,1,34.849750
5,75851,33249845,15.7,2011-11-05 00:22:00 UTC,-1340.648410,1644.421482,-3356.666300,872.697628,1,16409.239135
6,91422,23566642,16.1,2011-05-18 13:24:00 UTC,57.418457,1292.016128,1153.572603,-881.985513,1,2246.893301
7,103745,16826862,12.9,2011-10-14 19:04:00 UTC,-736.216667,40.767035,-73.982377,40.725562,1,4773.694612
8,139447,39981694,13.7,2012-01-20 11:50:00 UTC,-74.011042,40.709780,-73.983163,493.533332,4,10321.507661
9,144253,9421322,7.3,2009-08-26 11:55:00 UTC,-768.550000,40.757812,-73.997040,40.740007,1,2136.043688


In [71]:
extreme_values[["pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude"]]

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
4949,-748.016667,40.739957,-74.003570,40.734192
32549,-74.016055,40.715155,-737.916665,40.697862
48506,-735.200000,40.770092,-73.980187,40.765530
56617,-73.960828,404.433332,-73.988357,40.769037
61793,-73.951385,401.066667,-73.982110,40.754117
75851,-1340.648410,1644.421482,-3356.666300,872.697628
91422,57.418457,1292.016128,1153.572603,-881.985513
103745,-736.216667,40.767035,-73.982377,40.725562
139447,-74.011042,40.709780,-73.983163,493.533332
144253,-768.550000,40.757812,-73.997040,40.740007


Negative Longitude Values: Longitude values should typically fall within the range of -180 to 180 degrees. Values like -748.016667 and -736.400000 and so on are outside this range and need to be corrected.

Extreme Latitude Values: Latitude values should typically fall within the range of -90 to 90 degrees. Values like 404.433332, 401.066667, 1644.421482 and so on are outside this range and need to be corrected

In [72]:
costliest_ride = df[df['fare_amount'] == df['fare_amount'].max()]

# Extract pickup and dropoff coordinates
pickup_longitude = costliest_ride['pickup_longitude'].values[0]
pickup_latitude = costliest_ride['pickup_latitude'].values[0]
dropoff_longitude = costliest_ride['dropoff_longitude'].values[0]
dropoff_latitude = costliest_ride['dropoff_latitude'].values[0]


In [73]:
def haversine(lon1, lat1, lon2, lat2):
    # Convert coordinates from degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    r = 6371  # Radius of Earth in kilometers
    return r * c

In [74]:
haversine(pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude)

0.0007899213191009993

In [75]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [76]:
rides_in_2014 = df[df['pickup_datetime'].dt.year == 2014]
len(rides_in_2014)

29968

In [77]:
 len(df[(df['pickup_datetime'].dt.year == 2014) & (df['pickup_datetime'].dt.quarter == 1)])

7687

In [78]:
rides_in_september_2010 = df[(df['pickup_datetime'].dt.year == 2010) & (df['pickup_datetime'].dt.month == 9)]
day_of_week_counts = rides_in_september_2010['pickup_datetime'].dt.day_name().value_counts()


In [79]:
print(f"max_rides_day: {day_of_week_counts.idxmax()}, max_rides_count: {day_of_week_counts.max()}")


max_rides_day: Thursday, max_rides_count: 457



### Cleanning Data



In [80]:
min_longitude = -180
max_longitude = 180
min_latitude = -90
max_latitude = 90

filtered_df = df[(df['pickup_longitude'] >= min_longitude) &
                  (df['pickup_longitude'] <= max_longitude) &
                  (df['pickup_latitude'] >= min_latitude) &
                  (df['pickup_latitude'] <= max_latitude) &
                  (df['dropoff_longitude'] >= min_longitude) &
                  (df['dropoff_longitude'] <= max_longitude) &
                  (df['dropoff_latitude'] >= min_latitude) &
                  (df['dropoff_latitude'] <= max_latitude)]

In [81]:
filtered_df

,ride_id,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,haversine_distance
0,24238194,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323
1,27835199,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590
2,44984355,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377
3,25894730,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683
4,17610152,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450
...,...,...,...,...,...,...,...,...,...
199995,42598914,3.0,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,1,0.112210
199996,16382965,7.5,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1,1.875050
199997,27804658,30.9,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,2,12.850319
199998,20259894,14.5,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695415,1,3.539715


In [82]:
filtered_df.isna().sum()

ride_id               0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
passenger_count       0
haversine_distance    0
dtype: int64

In [83]:
filtered_df['pickup_datetime'] = pd.to_datetime(filtered_df['pickup_datetime'])

<ipython-input-83-6d66af74c63b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['pickup_datetime'] = pd.to_datetime(filtered_df['pickup_datetime'])


In [84]:
filtered_df['ride_week_day'] = filtered_df['pickup_datetime'].dt.day_name()

<ipython-input-84-5e27c1f06aeb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['ride_week_day'] = filtered_df['pickup_datetime'].dt.day_name()


In [85]:
filtered_df['ride_week_day']

0          Thursday
1            Friday
2            Monday
3            Friday
4          Thursday
            ...    
199995       Sunday
199996       Friday
199997       Monday
199998    Wednesday
199999     Saturday
Name: ride_week_day, Length: 199987, dtype: object

In [86]:
filtered_df.dtypes

ride_id                             int64
fare_amount                       float64
pickup_datetime       datetime64[ns, UTC]
pickup_longitude                  float64
pickup_latitude                   float64
dropoff_longitude                 float64
dropoff_latitude                  float64
passenger_count                     int64
haversine_distance                float64
ride_week_day                      object
dtype: object

In [87]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder


In [88]:
label_encoder = LabelEncoder()
filtered_df['ride_week_day'] = label_encoder.fit_transform(filtered_df['ride_week_day'])

<ipython-input-88-0fc0400af07e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['ride_week_day'] = label_encoder.fit_transform(filtered_df['ride_week_day'])


In [89]:
filtered_df['ride_week_day'].value_counts()

0    30878
2    30246
4    30021
6    29033
5    28126
3    26440
1    25243
Name: ride_week_day, dtype: int64

In [90]:
filtered_df['haversine_distance'].value_counts()

0.000000    5632
0.000424       3
0.000778       2
0.000334       2
0.000556       2
            ... 
5.128535       1
1.537437       1
3.642422       1
0.711464       1
5.417783       1
Name: haversine_distance, Length: 194349, dtype: int64

In [93]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

In [100]:
X = filtered_df[['passenger_count', 'haversine_distance',"ride_week_day"]]
y = filtered_df['fare_amount']

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [102]:
models = [
    ("Linear Regression", LinearRegression()),
    ("Decision Tree", DecisionTreeRegressor()),
    ("Random Forest", RandomForestRegressor()),
    ("Gradient Boosting", GradientBoostingRegressor())
]

In [103]:
results = []

In [104]:
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    results.append((name, r2))

results.sort(key=lambda x: x[1])

for name, r2 in results:
    print(f"{name}: Adjusted R-squared = {r2:.4f}")


Linear Regression: Adjusted R-squared = 0.0009
Decision Tree: Adjusted R-squared = 0.4485
Random Forest: Adjusted R-squared = 0.6654
Gradient Boosting: Adjusted R-squared = 0.7275


In [105]:
results

[('Linear Regression', 0.0008561157850560663),
 ('Decision Tree', 0.4484883012105335),
 ('Random Forest', 0.6654370243643801),
 ('Gradient Boosting', 0.7274970522638295)]